In [1]:
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import xarray as xr

import drdmannturb as drdmt


# CREATE DATASET IN DRDMANNTURB

In [2]:
domain = torch.logspace(-1, 3, 40)

L = 70
Gamma = 3.7
sigma = 0.04
Uref = 21

# NOTE: Below is obtained from the LES data... we used the 31st of 60
#       heights that we were given, so this is height[30]
zref = 148.56202535609793

In [3]:
spectra_file = Path("data_cleaned/log_downsampled_6component_spectra.dat")
CustomData = torch.tensor(np.genfromtxt(spectra_file, skip_header=1, delimiter = ","))

f = CustomData[:, 0]
k1_data_pts = 2 * torch.pi * f / Uref

gen = drdmt.OnePointSpectraDataGenerator(
    zref = zref,
    data_points = k1_data_pts,
    data_type = drdmt.DataType.CUSTOM,
    spectra_file = spectra_file,
    k1_data_points = k1_data_pts.data.cpu().numpy(),
)

gen.Data

([(tensor(0.0010, dtype=torch.float64), 148.56202535609793),
  (tensor(0.0015, dtype=torch.float64), 148.56202535609793),
  (tensor(0.0020, dtype=torch.float64), 148.56202535609793),
  (tensor(0.0025, dtype=torch.float64), 148.56202535609793),
  (tensor(0.0030, dtype=torch.float64), 148.56202535609793),
  (tensor(0.0035, dtype=torch.float64), 148.56202535609793),
  (tensor(0.0040, dtype=torch.float64), 148.56202535609793),
  (tensor(0.0045, dtype=torch.float64), 148.56202535609793),
  (tensor(0.0050, dtype=torch.float64), 148.56202535609793),
  (tensor(0.0055, dtype=torch.float64), 148.56202535609793),
  (tensor(0.0060, dtype=torch.float64), 148.56202535609793),
  (tensor(0.0065, dtype=torch.float64), 148.56202535609793),
  (tensor(0.0070, dtype=torch.float64), 148.56202535609793),
  (tensor(0.0075, dtype=torch.float64), 148.56202535609793),
  (tensor(0.0080, dtype=torch.float64), 148.56202535609793),
  (tensor(0.0085, dtype=torch.float64), 148.56202535609793),
  (tensor(0.0090, dtype=

# FIT 6 component spectra

In [4]:
domain = torch.logspace(-1, 3, 40)

L = 70
Gamma = 3.7
sigma = 0.04
Uref = 21

# NOTE: Below is obtained from the LES data... we used the 31st of 60
#       heights that we were given, so this is height[30]
zref = 148.56202535609793
device = "cpu"

pb = drdmt.CalibrationProblem(
    nn_params=drdmt.NNParameters(
        nlayers=3, hidden_layer_sizes=[10, 10, 10], activations=[nn.ReLU(), nn.ReLU(), nn.ReLU()]
    ),
    prob_params=drdmt.ProblemParameters(
        data_type=drdmt.DataType.CUSTOM, 
        tol=1e-9,
        nepochs=5,
        learn_nu=True,
        learning_rate=0.3, # Default is 0.1
        # init_with_noise=True,
        noise_magnitude=0.1 # Default is 0.01
    ),
    loss_params=drdmt.LossParameters(
        alpha_pen1=10.0,
        alpha_pen2=10.0,
        beta_reg=1e-1
    ),
    phys_params=drdmt.PhysicalParameters(
        L=L,
        Gamma=Gamma,
        sigma=sigma,
        domain=domain,
        Uref=Uref,
        zref=zref,
    ),
    logging_directory="runs/custom_data",
    device=device,
)

In [5]:
optimal_params = pb.calibrate(data=gen.Data)

Initial loss: nan


  0%|          | 0/5 [00:58<?, ?it/s]


RuntimeError: Loss is not a finite value, check initialization and learning hyperparameters.